# WFST i rozpoznawanie mowy

Zacznijemy od instalacji programów do systemu Kaldi i OpenFST:

In [ ]:
!pip install kora openfst-python arpa -q
import kora.install.kaldi

Dodajemy lmbin do ścieżki:

In [ ]:
import os
os.environ['PATH']+=':/opt/kaldi/src/lmbin'

Potrzebujemy też:
* ngram-count - program do trenowania modelu języka z pakietu SRILM
* ngram - program do modyfikacji modelu języka z pakietu SRILM

Oprócz tego potrebujemy następujące katalogi:
* model - model akustyczny (wytrenowany na nagraniach studyjnych)
* nagrania - kilka przykładowych plików audio
* make_lexicon.py - program generujący FST zawierający leksykon G2P
* sejm - pliki z modelem/nagraniami sejmowymi

## WFST

Importujemy moduł biblioteki OpenFST:

In [ ]:
import openfst_python as fst

### Przykład automatu

Zaczynamy od deninicji symboli wejściowych (A,B,C) i wyjśćiowych (I,II,III). Używając klasy `fst.SymbolTable` i metody `add_symbol`.

Tworzymy objekt `fst.Fst` i ustawiamy odpowiednio `set_input_symbols` oraz `set_output_symbols`.

Definiujemy 3 stany s0,s1,s2 metodą `add_state`.

Metodą `add_arc` dodajemy objekty `fst.Arc` losowo pomiędzy stanami.

Na samym końcu ustawiamy stan początkowy metodą `set_start` oraz stan(y) końcowe metodą `set_final`.

Teraz robimy specjalny automat zmieniający liczby rzymskie (I,II,III) na arabskie (1,2,3).

Teraz dokonamy kompozycji dwóch ostatnich automatów:


## Rozpoznawanie mowy

### Leksykon

Bierzemy następujący zestaw fonemów opisany na [tej](https://en.wikipedia.org/wiki/Polish_phonology) stronie.

Lista transkrypcji:
```
[('zero', ["z", "e", "r", "o"]),
 ('jeden', ["j", "e", "d", "e", "n"]),
 ('dwa', ["d", "v", "a"]),
 ('trzy', ["t", "S", "I"]),
 ('cztery', ["tS", "t", "e", "r", "I"]),
 ('pięć', ["p", "j", "e~", "ts'"]),
 ('pięć', ["p", "j", "e", "n'", "ts'"]),
 ('sześć', ["S", "e", "s'", "ts'"]),
 ('siedem', ["s'", "e", "d", "e", "m"]),
 ('osiem', ["o", "s'", "e", "m"]),
 ('dziewięć', ["dz'", "e", "v", "j", "e~", "ts'"]),
 ('dziewięć', ["dz'", "e", "v", "j", "e", "n'", "ts'"])]
```

Listy fonemów:

- nonsilence_phones - `["a","b","d","dz","dz'","dZ","e","e~","f","g","g'","G","i","I","j","k","k'","l","m","n","n'","N","o","o~","p","r","s","s'","S","t","ts","ts'","tS","u","v","w","x","x'","z","z'","Z"]`
- silence_phones - `['sil']`
- optional_silence - `'sil'`

Uruchamiamy funkcję `prepare_lexicon` z pliku `make_lexicon.py`:

Ustawianie symboli we/wy i wizualizacja:

### Gramatyka

Zobacz pliki:
* `words.txt`
* `phones.txt`
* `disambig.int`
* `word_boundary.int`

Tworzymy gramatykę do rozpoznawania ciągów cyfr, a potem robimy:
- `G.arcsort()`
- `fst.determinize(G)`
- `G.minimize()`

Losujemy zdania metodą `fst.randgen`:

Tworzymy własne zdanie:

Testujemy czy zdanie jest zgodne z gramatyką:

### Kompozycja automatu HCLG.fst

Łączymy automaty L i G i zapisujemy do pliku 'LG.fst':

Dodajemy kontekst trifonowy do modelu LG żeby otrzymać CLG:


In [ ]:
!fstcomposecontext --context-size=3 --central-position=1 --read-disambig-syms=disambig.int --write-disambig-syms=disambig_ilabels.int ilabels LG.fst CLG.fst

Tworzymy model łączenia stanów H:

In [ ]:
!make-h-transducer --transition-scale=1.0 --disambig-syms-out=disambig_tid.int ilabels model/tree model/final.mdl H.fst

Łaczymy H i CGL, a potem minimalizujemy i determinizujemy model:


In [ ]:
!fsttablecompose H.fst CLG.fst - | fstdeterminizestar --use-log=true - | fstrmsymbols disambig_tid.int - - | fstminimizeencoded - - | add-self-loops --self-loop-scale=1.0 --reorder=true model/final.mdl - HCLG.fst

### Ekstrakcja cech

Tworzymy plik `wav.scp`:

Liczymy MFCC programem `compute-mfcc-feats`:

- ustawiamy `--use-energy=false`

Liczymy nomralizację programem `compute-cmvn-stats`:

Stosujemy CMVN, dodajemy kontekstu z lewej i prawej strony, konwersja LDA:

* `apply-cmvn cmvn_stats ark:mfcc ark:-`
* `splice-feats --left-context=3 --right-context=3 ark:- ark:-`
* `transform-feats model/final.mat ark:- ark:feats`

### Dekodowanie

Program `gmm-decode-faster`:

Dokonujemy rozpoznawania programem `gmm-decode-faster`:

Dokonujemy mapowania słów z indeksów na stringi używając symboli `wsyms`:

Wyświetlamy poprawny wynik z pliku `nagrania/text`:

Liczymy WER programem `compute-wer`:

# Modelowanie języka

Robimy przykładowy korpus tekstowy:

Liczymy prosty model języka:

* `-text test.txt`
* `-order 3 `
* `-wbdiscount`

Wyświetlamy zawartość modelu języka:

Format tego pliku jest dosyć prosty i czytelny. Składa się z nagłówka zaczynającego od tokenu `/data/` i zawierającego liczność poszczególnych n-gramów. Potem mamy kolejne sekcje, każda zawierająca listę poszczególnych n-gramów.

Każdy n-gram jest opisany dwoma lub trzema polami oddzielonymi znakami `\t`:
* prawdopodobieństwo danego n-gramu w skali logarytmicznej
* opis samego n-gramu (tokeny/słowa oddzielone spacją)
* opcjonalnie tzw. "*back-off weight*" też w skali log

Back-off jest metodą do określenia prawdopodobieństwa n-gramów wyższego stopnia użwyając n-gramów niższego. Z tego powodu, najwyższe n-gramy (w naszym przypadku 3-gramy) nie mają policzonych wag back-off. Algorytm liczenia prawdopodonieństwa n-gramu jest następujący:

* jeśli na liście jest dokładnie ten n-gram którego szukamy, bierzemy jego prawdopodobieństwo
* jeśli go nie ma liście, bierzemy prawdopodobieństwo według wzoru:

\begin{equation}
P( word_N | word_{N-1}, word_{N-2}, ...., word_1 ) = \\
P( word_N | word_{N-1}, word_{N-2}, ...., word_2 ) \cdot \text{backoff-weight}(  word_{N-1} | word_{N-2}, ...., word_1 )
\end{equation}

* jeśli brakuje prawdopodobieństwa n-gramu mniejszego stopnia, wtedy rekurencyjnie stosujemy ten sam wzór aż do unigramów (które wszystkie powinny być zdefiniowane)
* jeśli brakuje wagi back-off, zakładmy wartość 1 (czyli 0 w skali logarytmicznej)

Na przykład, prawdopodobieństwo n-gramu "*ala ma*" jest następujące:

\begin{equation}
P(ma|ala) = 10^{-0.1760913} = 0.6666666038148176
\end{equation}

A prawdopodobieństwo n-gramu "*jan ma psa*":


\begin{equation}
P(psa|jan,ma) = P(psa|ma)*bwt(ma|jan)=10^{(-0.69897+0)}=0.20000000199681048
\end{equation}

Użyjemy prostej biblioteki `arpa` żeby potwierdzić powyższe obliczenia. Dokumentacja do biblioteki jest [tutaj](https://pypi.org/project/arpa/).

Robimy testowy plik tekstowy:

Liczymy perplexity:

Generujemy losowe zdania z modelu języka:

Konwertujemy model języka na G.fst:

## Dane sejmowe

Trenujemy model języka na pliku `sejm/text`:
* `-order 3`
* `-unk`
* `-kndiscount`
* `-text sejm/text`
* `-write-vocab words.list`

Generujemy losowe zdania z tego modelu:

Liczymy perplexity tranksrypcji nagrań:

## WFST

Generujemy `G.fst`:

In [ ]:
!gunzip -c sejm.arpa.gz | arpa2fst --disambig-symbol=#0 --read-symbol-table=sejm/lang/words.txt - G.fst

Generujemy `HCLG.fst`:

In [ ]:
!fsttablecompose sejm/lang/L_disambig.fst G.fst | fstdeterminizestar --use-log=true | fstminimizeencoded | fstpushspecial > LG.fst
!fstcomposecontext --context-size=3 --central-position=1 --read-disambig-syms=sejm/lang/phones/disambig.int --write-disambig-syms=disambig_ilabels.int ilabels LG.fst | fstarcsort --sort_type=ilabel > CLG.fst
!make-h-transducer --disambig-syms-out=disambig_tid.int --transition-scale=1.0 ilabels sejm/model/tree sejm/model/final.mdl H.fst
!fsttablecompose H.fst CLG.fst - | fstdeterminizestar --use-log=true - - | fstrmsymbols disambig_tid.int - - | fstminimizeencoded - - | add-self-loops --self-loop-scale=0.1 --reorder=true sejm/model/final.mdl - HCLG.fst

## Ekstrakcja cech

Liczymy cechy z katalogu `sejm/audio`:

## Rozpoznawanie

Uruchamiamy dekoder:

Konwertujemy plik wyjściowy z indeksów na tekst:

Liczymy końcowy WER:

# Przykładowe zadania

* przygotuj własne nagrania do przetestowania rozpoznawania mowy
* przetestuj wpływ różnych parametrów dekodowania na prędkość i jakość rozpoznawania
* wymyśl własną gramatykę i stwórz system rozpoznawania mowy, który będzie rozpoznawał zdania z tej gramatyki
* przygotuj korpus tekstów, wytrenuj model języka i stwórz model do rozpoznawania zdań z tego modelu
* wygeneruj kratę programem `gmm-latgen-faster` i narysuj jej zawartość
* przetestuj metody rescoringu krat i liczenia oracle